# Daily Knowledge Bot Tutorial

This tutorial guides you through creating a simple application that uses the Perplexity API to fetch an interesting fact on a different topic each day.

## What You'll Learn
- How to authenticate with the Perplexity API
- How to structure API requests and handle responses
- How to implement a simple topic rotation system
- How to save results to files

## Prerequisites
- A Perplexity API key
- Python 3.6 or higher
- The `requests` library installed

## 1. Setting Up Your Environment

First, let's import the necessary libraries and set up our API key.

In [ ]:
import requests
import json
import os
from datetime import datetime

# Replace this with your actual Perplexity API key
API_KEY = "your_perplexity_api_key"

# Alternatively, you can set it as an environment variable
# API_KEY = os.environ.get("PERPLEXITY_API_KEY")

# Verify we have an API key
if not API_KEY or API_KEY == "your_perplexity_api_key":
    print("⚠️ Warning: You need to set your actual API key to use this notebook.")
else:
    print("✅ API key is set.")

## 2. Creating the API Request Function

Next, let's create a function that calls the Perplexity API to get an interesting fact about a given topic.

In [ ]:
def get_daily_fact(topic):
    """
    Fetches an interesting fact about the given topic using Perplexity API.
    
    Args:
        topic (str): The topic to get a fact about
        
    Returns:
        str: An interesting fact about the topic
    """
    url = "https://api.perplexity.ai/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "sonar",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant that provides interesting, accurate, and concise facts. Respond with only one fascinating fact, kept under 100 words."
            },
            {
                "role": "user",
                "content": f"Tell me an interesting fact about {topic} that most people don't know."
            }
        ],
        "max_tokens": 150,
        "temperature": 0.7
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for 4XX/5XX responses
        
        result = response.json()
        return result["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        return f"Error making API request: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"Error parsing API response: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

### Let's test the function with a sample topic

In [ ]:
# Try getting a fact about astronomy
test_fact = get_daily_fact("astronomy")
print(test_fact)

## 3. Creating a File Saving Function

Now, let's create a function to save our fact to a text file so we can keep a record of all the facts we've learned.

In [ ]:
def save_fact_to_file(topic, fact):
    """
    Saves the fact to a text file with timestamp.
    
    Args:
        topic (str): The topic of the fact
        fact (str): The fact content
        
    Returns:
        str: Path to the saved file
    """
    timestamp = datetime.now().strftime("%Y-%m-%d")
    filename = f"daily_fact_{timestamp}.txt"
    
    with open(filename, "w") as f:
        f.write(f"DAILY FACT - {timestamp}\n")
        f.write(f"Topic: {topic}\n\n")
        f.write(fact)
    
    print(f"Fact saved to {filename}")
    return filename

### Test saving a fact to a file

In [ ]:
# Test saving our astronomy fact
saved_file = save_fact_to_file("astronomy", test_fact)

# Let's read the file to verify it worked
with open(saved_file, 'r') as f:
    print(f.read())

## 4. Creating the Topic Rotation Function

We want our bot to provide facts on different topics. Let's create a function that selects a topic based on the current day of the month.

In [ ]:
def get_todays_topic():
    """
    Selects a topic based on the current day of the month.
    
    Returns:
        str: Today's selected topic
    """
    topics = [
        "astronomy", 
        "history", 
        "biology", 
        "technology", 
        "psychology",
        "ocean life",
        "ancient civilizations",
        "quantum physics",
        "art history",
        "culinary science",
        "linguistics",
        "architecture",
        "mythology",
        "mathematics",
        "geography",
        "music theory",
        "cryptocurrency",
        "neuroscience",
        "climate science",
        "space exploration",
        "anthropology",
        "philosophy",
        "artificial intelligence",
        "genetics",
        "economics",
        "literature",
        "chemistry",
        "geology",
        "zoology",
        "sustainable energy",
        "robotics"
    ]
    
    # Get the current day of the month (1-31)
    day = datetime.now().day
    
    # Use modulo to wrap around if we have more days than topics
    topic_index = (day % len(topics)) - 1
    if topic_index < 0:  # Handle the case where day % len(topics) = 0
        topic_index = 0
        
    today_topic = topics[topic_index]
    print(f"Today's topic is: {today_topic}")
    return today_topic

# Let's test it
today_topic = get_todays_topic()

## 5. Putting It All Together

Now, let's create our main function that combines all of these components.

In [ ]:
def run_daily_knowledge_bot():
    """
    Main function that runs the daily knowledge bot.
    Gets a fact about today's topic and saves it to a file.
    """
    # Get today's topic
    today_topic = get_todays_topic()
    print(f"Getting today's fact about: {today_topic}")
    
    # Get the fact
    fact = get_daily_fact(today_topic)
    print(f"\nToday's {today_topic} fact: {fact}")
    
    # Save the fact to a file
    saved_file = save_fact_to_file(today_topic, fact)
    
    return {
        "topic": today_topic,
        "fact": fact,
        "file": saved_file,
        "date": datetime.now().strftime("%Y-%m-%d")
    }

### Run the bot!

In [ ]:
# Run our daily knowledge bot
result = run_daily_knowledge_bot()

# Display the result in a more structured way
print("\n" + "-"*50)
print("DAILY KNOWLEDGE BOT RESULT")
print("-"*50)
print(f"Date: {result['date']}")
print(f"Topic: {result['topic']}")
print(f"Fact saved to: {result['file']}")
print("-"*50)

## 6. Advanced: Other Ways to Share the Daily Fact

Saving to a file is just one way to store our facts. Here are some additional methods we could implement:

### Email the daily fact

Here's how you could send the fact via email using the `smtplib` library:

In [ ]:
def send_email(topic, fact, recipient):
    """
    Sends the daily fact via email.
    
    Args:
        topic (str): The topic of the fact
        fact (str): The fact content
        recipient (str): Email address to send to
        
    Note: This is just a template. You would need to fill in your SMTP server details.
    """
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    
    # Email details
    sender_email = "your_email@example.com"  # Update this
    password = "your_password"  # Update this (consider using app passwords for security)
    
    # Create message
    message = MIMEMultipart()
    message["Subject"] = f"Daily Knowledge: {topic.capitalize()}"
    message["From"] = sender_email
    message["To"] = recipient
    
    # Email body
    current_date = datetime.now().strftime("%Y-%m-%d")
    email_content = f"""
    <html>
    <body>
        <h2>Your Daily Interesting Fact - {current_date}</h2>
        <h3>Topic: {topic.capitalize()}</h3>
        <p>{fact}</p>
        <hr>
        <p><em>Brought to you by Daily Knowledge Bot</em></p>
    </body>
    </html>
    """
    
    message.attach(MIMEText(email_content, "html"))
    
    # Connect to server and send (commented out as this requires actual credentials)
    """
    with smtplib.SMTP_SSL("smtp.example.com", 465) as server:  # Update with your SMTP server
        server.login(sender_email, password)
        server.send_message(message)
    print(f"Email sent to {recipient}")
    """
    
    # Since we're just displaying the concept, print what would be sent
    print(f"\nWould send email to: {recipient}")
    print(f"Subject: Daily Knowledge: {topic.capitalize()}")
    print("Content:")
    print(f"Topic: {topic.capitalize()}")
    print(f"Fact: {fact}")

### Save to a database

If you wanted to build up a collection of facts over time, saving to a database would be ideal. Here's a simple example using SQLite:

In [ ]:
def save_to_database(topic, fact):
    """
    Saves the fact to an SQLite database.
    
    Args:
        topic (str): The topic of the fact
        fact (str): The fact content
    """
    import sqlite3
    from datetime import datetime
    
    # Connect to database (creates it if it doesn't exist)
    conn = sqlite3.connect("daily_facts.db")
    cursor = conn.cursor()
    
    # Create table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS facts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        topic TEXT,
        fact TEXT
    )
    """)
    
    # Insert the new fact
    current_date = datetime.now().strftime("%Y-%m-%d")
    cursor.execute(
        "INSERT INTO facts (date, topic, fact) VALUES (?, ?, ?)",
        (current_date, topic, fact)
    )
    
    # Commit and close
    conn.commit()
    conn.close()
    
    print(f"Fact saved to database with topic '{topic}'")

## 7. Setting Up for Daily Execution

To make this bot truly useful, we'd want it to run automatically every day. Here are some options for scheduling:

### Cron Jobs (Linux/Mac)

If you're using Linux or Mac, you can use cron to schedule the script to run daily:

1. Save the complete script as `daily_knowledge_bot.py`
2. Make it executable with: `chmod +x daily_knowledge_bot.py`
3. Edit your crontab with: `crontab -e`
4. Add a line like this to run it at 9 AM daily:
   ```
   0 9 * * * /path/to/python /path/to/daily_knowledge_bot.py
   ```

### Task Scheduler (Windows)

If you're using Windows:

1. Save the script as `daily_knowledge_bot.py`
2. Open Task Scheduler
3. Create a Basic Task
4. Set it to run daily
5. Set the action to "Start a Program"
6. Browse to your Python executable and add the script as an argument
   ```
   Program: C:\Path\to\Python.exe
   Arguments: C:\Path\to\daily_knowledge_bot.py
   ```

## 8. Complete Script

Here's the complete script that you could save as `daily_knowledge_bot.py` for daily execution:

In [ ]:
# Below is the complete script you can save as daily_knowledge_bot.py

#!/usr/bin/env python3
"""
Daily Knowledge Bot

This script uses the Perplexity API to fetch an interesting fact about a rotating
topic each day. It can be scheduled to run daily using cron or Task Scheduler.

Usage:
  python daily_knowledge_bot.py

Requirements:
  - requests
"""

import requests
import json
import os
from datetime import datetime

# Store your API key securely (consider using environment variables in production)
# API_KEY = os.environ.get("PERPLEXITY_API_KEY")
API_KEY = "your_perplexity_api_key"  # Replace with your actual API key


def get_daily_fact(topic):
    """
    Fetches an interesting fact about the given topic using Perplexity API.
    
    Args:
        topic (str): The topic to get a fact about
        
    Returns:
        str: An interesting fact about the topic
    """
    url = "https://api.perplexity.ai/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "sonar",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant that provides interesting, accurate, and concise facts. Respond with only one fascinating fact, kept under 100 words."
            },
            {
                "role": "user",
                "content": f"Tell me an interesting fact about {topic} that most people don't know."
            }
        ],
        "max_tokens": 150,
        "temperature": 0.7
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for 4XX/5XX responses
        
        result = response.json()
        return result["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        return f"Error making API request: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"Error parsing API response: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"


def save_fact_to_file(topic, fact):
    """
    Saves the fact to a text file with timestamp.
    
    Args:
        topic (str): The topic of the fact
        fact (str): The fact content
    """
    timestamp = datetime.now().strftime("%Y-%m-%d")
    filename = f"daily_fact_{timestamp}.txt"
    
    with open(filename, "w") as f:
        f.write(f"DAILY FACT - {timestamp}\n")
        f.write(f"Topic: {topic}\n\n")
        f.write(fact)
    
    print(f"Fact saved to {filename}")


def main():
    """
    Main function that runs the daily knowledge bot.
    """
    # List of topics to rotate through
    topics = [
        "astronomy", 
        "history", 
        "biology", 
        "technology", 
        "psychology",
        "ocean life",
        "ancient civilizations",
        "quantum physics",
        "art history",
        "culinary science",
        "linguistics",
        "architecture",
        "mythology",
        "mathematics",
        "geography",
        "music theory",
        "cryptocurrency",
        "neuroscience",
        "climate science",
        "space exploration",
        "anthropology",
        "philosophy",
        "artificial intelligence",
        "genetics",
        "economics",
        "literature",
        "chemistry",
        "geology",
        "zoology",
        "sustainable energy",
        "robotics"
    ]
    
    # Use the current day of month to select a topic (rotates through the list)
    day = datetime.now().day
    topic_index = (day % len(topics)) - 1
    if topic_index < 0:  # Handle the case where day % len(topics) = 0
        topic_index = 0
    today_topic = topics[topic_index]
    
    print(f"Getting today's fact about: {today_topic}")
    
    # Get and display the fact
    fact = get_daily_fact(today_topic)
    print(f"\nToday's {today_topic} fact: {fact}")
    
    # Save the fact to a file
    save_fact_to_file(today_topic, fact)
    
    # In a real application, you might send this via email, SMS, etc.
    # Example: send_email("Daily Interesting Fact", fact, "your@email.com")


if __name__ == "__main__":
    main()